<a href="https://colab.research.google.com/github/SanmeetGulati/Big-Data-Analytics-Basics/blob/main/BDA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [2]:
pip install findspark

In [3]:
import findspark
findspark.init()
import pyspark
sc=pyspark.SparkContext(appName="MyAppName")
sc

<SparkContext master=local[*] appName=MyAppName>

# Question 1: Student and Class DataFrames
Create a student dataframe with following details:-

student(rollno , name, address, age, gender)

class(classid, classname)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit

spark = SparkSession.builder.appName("Assignment1_Q1").getOrCreate()

student_data = [
    (1, "Isha", "Pune", 22, "F"),
    (2, "Divya", "Mumbai", 22, "F"),
    (3, "Harsh", "Pune", 24, "M"),
    (4, "Sanmeet", "Pune", 21, "F"),
    (5, "Gulafsha", "Mumbai", 21, "F"),
]
class_data = [
    (101, "Data Science"),
    (102, "ML"),
]
student_schema = ["rollno", "name", "address", "age", "gender"]
class_schema = ["classid", "classname"]


student_df = spark.createDataFrame(student_data, student_schema)
class_df = spark.createDataFrame(class_data, class_schema)


Display DataFrames


In [5]:
student_df.show()
class_df.show()

+------+--------+-------+---+------+
|rollno|    name|address|age|gender|
+------+--------+-------+---+------+
|     1|    Isha|   Pune| 22|     F|
|     2|   Divya| Mumbai| 22|     F|
|     3|   Harsh|   Pune| 24|     M|
|     4| Sanmeet|   Pune| 21|     F|
|     5|Gulafsha| Mumbai| 21|     F|
+------+--------+-------+---+------+

+-------+------------+
|classid|   classname|
+-------+------------+
|    101|Data Science|
|    102|          ML|
+-------+------------+



Display student names with gender M and F separately

In [6]:
print("Male students:")
student_df.filter(col("gender") == "M").select("name").show()
print("Female students:")
student_df.filter(col("gender") == "F").select("name").show()

Male students:
+-----+
| name|
+-----+
|Harsh|
+-----+

Female students:
+--------+
|    name|
+--------+
|    Isha|
|   Divya|
| Sanmeet|
|Gulafsha|
+--------+



 Display students of a specific class (need mapping, demo adding classid=101 for all)

In [7]:
student_df = student_df.withColumn("classid", lit(101))
student_df.join(class_df, "classid").show()

+-------+------+--------+-------+---+------+------------+
|classid|rollno|    name|address|age|gender|   classname|
+-------+------+--------+-------+---+------+------------+
|    101|     1|    Isha|   Pune| 22|     F|Data Science|
|    101|     2|   Divya| Mumbai| 22|     F|Data Science|
|    101|     3|   Harsh|   Pune| 24|     M|Data Science|
|    101|     4| Sanmeet|   Pune| 21|     F|Data Science|
|    101|     5|Gulafsha| Mumbai| 21|     F|Data Science|
+-------+------+--------+-------+---+------+------------+



Students whose age > 22

In [8]:
student_df.filter(col("age") > 22).show()

+------+-----+-------+---+------+-------+
|rollno| name|address|age|gender|classid|
+------+-----+-------+---+------+-------+
|     3|Harsh|   Pune| 24|     M|    101|
+------+-----+-------+---+------+-------+



Add column 'grade' to student frame

In [9]:
student_df = student_df.withColumn("grade", when(col("age") > 23, "A").otherwise("B"))
student_df.show()

+------+--------+-------+---+------+-------+-----+
|rollno|    name|address|age|gender|classid|grade|
+------+--------+-------+---+------+-------+-----+
|     1|    Isha|   Pune| 22|     F|    101|    B|
|     2|   Divya| Mumbai| 22|     F|    101|    B|
|     3|   Harsh|   Pune| 24|     M|    101|    A|
|     4| Sanmeet|   Pune| 21|     F|    101|    B|
|     5|Gulafsha| Mumbai| 21|     F|    101|    B|
+------+--------+-------+---+------+-------+-----+



Display classnames and associated students

In [10]:
student_df.join(class_df, "classid").select("classname", "name").show()

+------------+--------+
|   classname|    name|
+------------+--------+
|Data Science|    Isha|
|Data Science|   Divya|
|Data Science|   Harsh|
|Data Science| Sanmeet|
|Data Science|Gulafsha|
+------------+--------+



# Question 2: Employee and Department DataFrames


create dataframe

In [11]:
from pyspark.sql.functions import collect_list


emp_data = [
    (1, "Harsh", "M", "Manager", "Pune", 30000, 10),
    (2, "Isha", "F", "Analyst", "Mumbai", 20000, 20),
    (3, "Ujjwal", "M", "Clerk", "Pune", 15000, 10),
    (4, "Gulafsha", "F", "Manager", "Mumbai", 31000, 20),
    (5, "Harry", "M", "Analyst", "Pune", 21000, 10),
]
dept_data = [
    (10, "IT"),
    (20, "HR"),
    (30, "Finance"),
    (40, "Operations"),
    (50, "Legal"),
]
emp_schema = ["eno", "ename", "gender", "designation", "city", "salary", "dno"]
dept_schema = ["dno", "dname"]

emp_df = spark.createDataFrame(emp_data, emp_schema)
dept_df = spark.createDataFrame(dept_data, dept_schema)

Show Schemas

In [12]:
emp_df.show()
dept_df.show()

+---+--------+------+-----------+------+------+---+
|eno|   ename|gender|designation|  city|salary|dno|
+---+--------+------+-----------+------+------+---+
|  1|   Harsh|     M|    Manager|  Pune| 30000| 10|
|  2|    Isha|     F|    Analyst|Mumbai| 20000| 20|
|  3|  Ujjwal|     M|      Clerk|  Pune| 15000| 10|
|  4|Gulafsha|     F|    Manager|Mumbai| 31000| 20|
|  5|   Harry|     M|    Analyst|  Pune| 21000| 10|
+---+--------+------+-----------+------+------+---+

+---+----------+
|dno|     dname|
+---+----------+
| 10|        IT|
| 20|        HR|
| 30|   Finance|
| 40|Operations|
| 50|     Legal|
+---+----------+



Filter by designation, salary

In [13]:
emp_df.filter(col("designation") == "Manager").show()
emp_df.filter(col("salary") > 20000).show()

+---+--------+------+-----------+------+------+---+
|eno|   ename|gender|designation|  city|salary|dno|
+---+--------+------+-----------+------+------+---+
|  1|   Harsh|     M|    Manager|  Pune| 30000| 10|
|  4|Gulafsha|     F|    Manager|Mumbai| 31000| 20|
+---+--------+------+-----------+------+------+---+

+---+--------+------+-----------+------+------+---+
|eno|   ename|gender|designation|  city|salary|dno|
+---+--------+------+-----------+------+------+---+
|  1|   Harsh|     M|    Manager|  Pune| 30000| 10|
|  4|Gulafsha|     F|    Manager|Mumbai| 31000| 20|
|  5|   Harry|     M|    Analyst|  Pune| 21000| 10|
+---+--------+------+-----------+------+------+---+



Data of departments for female employees


In [14]:
emp_df.filter(col("gender") == "F").join(dept_df, "dno").show()

+---+---+--------+------+-----------+------+------+-----+
|dno|eno|   ename|gender|designation|  city|salary|dname|
+---+---+--------+------+-----------+------+------+-----+
| 20|  2|    Isha|     F|    Analyst|Mumbai| 20000|   HR|
| 20|  4|Gulafsha|     F|    Manager|Mumbai| 31000|   HR|
+---+---+--------+------+-----------+------+------+-----+



Increase salary of 'Manager' by 10%

In [15]:
emp_df = emp_df.withColumn("salary", when(emp_df.designation == "Manager", emp_df.salary * 1.1).otherwise(emp_df.salary))
emp_df.show()

+---+--------+------+-----------+------+-------+---+
|eno|   ename|gender|designation|  city| salary|dno|
+---+--------+------+-----------+------+-------+---+
|  1|   Harsh|     M|    Manager|  Pune|33000.0| 10|
|  2|    Isha|     F|    Analyst|Mumbai|20000.0| 20|
|  3|  Ujjwal|     M|      Clerk|  Pune|15000.0| 10|
|  4|Gulafsha|     F|    Manager|Mumbai|34100.0| 20|
|  5|   Harry|     M|    Analyst|  Pune|21000.0| 10|
+---+--------+------+-----------+------+-------+---+



Add more records and show updated schema

In [16]:
more_emp_data = [
    (6, "Divya", "F", "Analyst", "Pune", 22000, 30),
    (7, "Anuj", "M", "Clerk", "Mumbai", 14000, 40),
    (8, "Mayuri", "F", "Manager", "Pune", 32000, 30),
]
more_dept_data = [
    (60, "Marketing"),
    (70, "Sales"),
    (80, "Support"),
]
emp_df = emp_df.union(spark.createDataFrame(more_emp_data, emp_schema))
dept_df = dept_df.union(spark.createDataFrame(more_dept_data, dept_schema))
emp_df.show()
dept_df.show()

+---+--------+------+-----------+------+-------+---+
|eno|   ename|gender|designation|  city| salary|dno|
+---+--------+------+-----------+------+-------+---+
|  1|   Harsh|     M|    Manager|  Pune|33000.0| 10|
|  2|    Isha|     F|    Analyst|Mumbai|20000.0| 20|
|  3|  Ujjwal|     M|      Clerk|  Pune|15000.0| 10|
|  4|Gulafsha|     F|    Manager|Mumbai|34100.0| 20|
|  5|   Harry|     M|    Analyst|  Pune|21000.0| 10|
|  6|   Divya|     F|    Analyst|  Pune|22000.0| 30|
|  7|    Anuj|     M|      Clerk|Mumbai|14000.0| 40|
|  8|  Mayuri|     F|    Manager|  Pune|32000.0| 30|
+---+--------+------+-----------+------+-------+---+

+---+----------+
|dno|     dname|
+---+----------+
| 10|        IT|
| 20|        HR|
| 30|   Finance|
| 40|Operations|
| 50|     Legal|
| 60| Marketing|
| 70|     Sales|
| 80|   Support|
+---+----------+



Show join usage for unique records


In [17]:
emp_df.join(dept_df, "dno").dropDuplicates().show()

+---+---+--------+------+-----------+------+-------+----------+
|dno|eno|   ename|gender|designation|  city| salary|     dname|
+---+---+--------+------+-----------+------+-------+----------+
| 10|  1|   Harsh|     M|    Manager|  Pune|33000.0|        IT|
| 10|  3|  Ujjwal|     M|      Clerk|  Pune|15000.0|        IT|
| 10|  5|   Harry|     M|    Analyst|  Pune|21000.0|        IT|
| 20|  2|    Isha|     F|    Analyst|Mumbai|20000.0|        HR|
| 20|  4|Gulafsha|     F|    Manager|Mumbai|34100.0|        HR|
| 30|  6|   Divya|     F|    Analyst|  Pune|22000.0|   Finance|
| 30|  8|  Mayuri|     F|    Manager|  Pune|32000.0|   Finance|
| 40|  7|    Anuj|     M|      Clerk|Mumbai|14000.0|Operations|
+---+---+--------+------+-----------+------+-------+----------+



Department-wise list of employees


In [18]:
emp_df.join(dept_df, "dno").groupBy("dname").agg(collect_list("ename").alias("employees")).show()

+----------+--------------------+
|     dname|           employees|
+----------+--------------------+
|        HR|    [Isha, Gulafsha]|
|   Finance|     [Divya, Mayuri]|
|        IT|[Harsh, Ujjwal, H...|
|Operations|              [Anuj]|
+----------+--------------------+



Employees with salary < 20000 and designation == "Clerk"

In [19]:
emp_df.filter((col("salary") < 20000) & (col("designation") == "Clerk")).show()

+---+------+------+-----------+------+-------+---+
|eno| ename|gender|designation|  city| salary|dno|
+---+------+------+-----------+------+-------+---+
|  3|Ujjwal|     M|      Clerk|  Pune|15000.0| 10|
|  7|  Anuj|     M|      Clerk|Mumbai|14000.0| 40|
+---+------+------+-----------+------+-------+---+



# Question 3: Product, Customer, Orders DataFrames

Create DataFrames

In [21]:
from pyspark.sql.functions import sum as _sum, min as _min, max as _max, count as _count

product_data = [
    (1, "Pen", "Stationery", 15),
    (2, "Notebook", "Stationery", 45),
    (3, "Bottle", "Accessory", 90),
]
customer_data = [
    (1, "Ankita", "990", "Pune"),
    (2, "Priya", "880", "Mumbai"),
    (3, "Ravi", "770", "Pune"),
]
orders_data = [
    (1, "2025-08-01", 1, "COMPLETE"),
    (2, "2025-08-15", 2, "CLOSED"),
    (3, "2025-07-15", 3, "PENDING"),
]
order_items_data = [
    (1, 1, 2, 30),
    (1, 2, 1, 45),
    (2, 2, 2, 90),
    (3, 3, 1, 90),
]
prod_schema = ["product_id", "pname", "ptype", "price"]
cust_schema = ["cust_id", "cname", "mobileno", "city"]
orders_schema = ["order_id", "order_date", "order_customer_id", "order_status"]
order_items_schema = ["order_item_order_id", "order_item_product_id", "order_item_quantity", "order_item_subtotal"]

product_df = spark.createDataFrame(product_data, prod_schema)
customer_df = spark.createDataFrame(customer_data, cust_schema)
orders_df = spark.createDataFrame(orders_data, orders_schema)
order_items_df = spark.createDataFrame(order_items_data, order_items_schema)


 Get details of all customers from Pune city

In [22]:
customer_df.filter(col("city") == "Pune").show()


+-------+------+--------+----+
|cust_id| cname|mobileno|city|
+-------+------+--------+----+
|      1|Ankita|     990|Pune|
|      3|  Ravi|     770|Pune|
+-------+------+--------+----+



Orders with subtotal > 50 in August

In [23]:
orders_aug = orders_df.join(order_items_df, orders_df.order_id == order_items_df.order_item_order_id)\
    .filter((col("order_item_subtotal") > 50) & (col("order_date").substr(6, 2) == "08"))
orders_aug.show()

+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+
|       2|2025-08-15|                2|      CLOSED|                  2|                    2|                  2|                 90|
+--------+----------+-----------------+------------+-------------------+---------------------+-------------------+-------------------+



Orders in ascending order of subtotal

In [24]:
order_items_df.orderBy(col("order_item_subtotal").asc()).show()

+-------------------+---------------------+-------------------+-------------------+
|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|
+-------------------+---------------------+-------------------+-------------------+
|                  1|                    1|                  2|                 30|
|                  1|                    2|                  1|                 45|
|                  2|                    2|                  2|                 90|
|                  3|                    3|                  1|                 90|
+-------------------+---------------------+-------------------+-------------------+



Customer details with min/max order amount

In [25]:
order_totals = order_items_df.groupBy("order_item_order_id").agg(_sum("order_item_subtotal").alias("total"))
min_amt = order_totals.agg(_min("total")).first()["min(total)"]
max_amt = order_totals.agg(_max("total")).first()["max(total)"]

orders_df.join(order_totals, orders_df.order_id == order_totals.order_item_order_id)\
    .filter(col("total") == min_amt).join(customer_df, orders_df.order_customer_id == customer_df.cust_id).show()
orders_df.join(order_totals, orders_df.order_id == order_totals.order_item_order_id)\
    .filter(col("total") == max_amt).join(customer_df, orders_df.order_customer_id == customer_df.cust_id).show()


+--------+----------+-----------------+------------+-------------------+-----+-------+------+--------+----+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|total|cust_id| cname|mobileno|city|
+--------+----------+-----------------+------------+-------------------+-----+-------+------+--------+----+
|       1|2025-08-01|                1|    COMPLETE|                  1|   75|      1|Ankita|     990|Pune|
+--------+----------+-----------------+------------+-------------------+-----+-------+------+--------+----+

+--------+----------+-----------------+------------+-------------------+-----+-------+-----+--------+------+
|order_id|order_date|order_customer_id|order_status|order_item_order_id|total|cust_id|cname|mobileno|  city|
+--------+----------+-----------------+------------+-------------------+-----+-------+-----+--------+------+
|       3|2025-07-15|                3|     PENDING|                  3|   90|      3| Ravi|     770|  Pune|
|       2|2025-08-15|  

Orders which are COMPLETE or CLOSED

In [26]:
orders_df.filter(col("order_status").isin("COMPLETE", "CLOSED")).show()

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|       1|2025-08-01|                1|    COMPLETE|
|       2|2025-08-15|                2|      CLOSED|
+--------+----------+-----------------+------------+



Orders COMPLETE or CLOSED in August 2025

In [27]:
orders_df.filter(col("order_status").isin("COMPLETE", "CLOSED")).filter(col("order_date").startswith("2025-08")).show()

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|       1|2025-08-01|                1|    COMPLETE|
|       2|2025-08-15|                2|      CLOSED|
+--------+----------+-----------------+------------+



order_items where subtotal is not equal to quantity * price

In [28]:
order_items_df.join(product_df, order_items_df.order_item_product_id == product_df.product_id)\
    .filter(col("order_item_subtotal") != col("order_item_quantity") * col("price")).show()

+-------------------+---------------------+-------------------+-------------------+----------+-----+-----+-----+
|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|product_id|pname|ptype|price|
+-------------------+---------------------+-------------------+-------------------+----------+-----+-----+-----+
+-------------------+---------------------+-------------------+-------------------+----------+-----+-----+-----+



Orders placed on first of every month

In [29]:
orders_df.filter(col("order_date").substr(9, 2) == "01").show()


+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
|       1|2025-08-01|                1|    COMPLETE|
+--------+----------+-----------------+------------+



 Count by status from orders

In [30]:
orders_df.groupBy("order_status").agg(_count("order_id")).show()


+------------+---------------+
|order_status|count(order_id)|
+------------+---------------+
|    COMPLETE|              1|
|      CLOSED|              1|
|     PENDING|              1|
+------------+---------------+



Revenue for each order id from order_items

In [31]:
order_items_df.groupBy("order_item_order_id").agg(_sum("order_item_subtotal").alias("revenue")).show()


+-------------------+-------+
|order_item_order_id|revenue|
+-------------------+-------+
|                  1|     75|
|                  3|     90|
|                  2|     90|
+-------------------+-------+



Daily product revenue

In [32]:
orders_df.join(order_items_df, orders_df.order_id == order_items_df.order_item_order_id)\
    .groupBy("order_date", "order_item_product_id")\
    .agg(_sum("order_item_subtotal").alias("daily_revenue")).show()

+----------+---------------------+-------------+
|order_date|order_item_product_id|daily_revenue|
+----------+---------------------+-------------+
|2025-08-01|                    1|           30|
|2025-08-01|                    2|           45|
|2025-08-15|                    2|           90|
|2025-07-15|                    3|           90|
+----------+---------------------+-------------+

